In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

system_prompt_nl = """
You are a Fleet Maintenance & Operations Expert and AI Assistant. Your job is to convert business goals, written in plain language, into a weighted KPI scorecard.
 
You will be given:
 
- A list of KPIs (with labels that match internal data fields)
- A human-written business requirements statement
 
To improve your understanding, here is the internal reference for KPI definitions and formulas:
 
1. Avg_Maintenance_Working_Time_Hours: The average duration of maintenance work orders of a vehicle per year.
   Formula: Σ(DOWNTIME_HRS_SHOP) / Count(UNIQUE_WORK_ORDER_NO)
 
2. Total_Cost: Total annual maintenance cost per vehicle.
   Formula: Σ(TOTAL_COST), grouped by EQ_EQUIP_NO
 
3. Unscheduled_Repairs: Total number of unscheduled repair orders.
   Formula: Count where JOB_TYPE == 'REPAIR'
 
4. MTBF_Days: Mean time between failures — average time between two breakdowns.
   Formula: Σ(CREATE_DATE of JOB_TYPE == 'REPAIR' - previous CREATE_DATE) / Unscheduled Repair Count
 
5. Total_PM: Total number of preventive maintenance orders.
   Formula: Count where JOB_TYPE == 'PM'
 
6. Avg_Distance_Post_Maint: The average distance traveled after each maintenance of a vehicle.
   Formula: Σ(METER_1_READING difference) / Count(UNIQUE_WORK_ORDER_NO)
 
7. Avg_Downtime_Hours: Average downtime of a vehicle per year.
   Formula: Σ(DATETIME_FINISHED - DATETIME_OUT_SERVICE) / Count(UNIQUE_WORK_ORDER_NO)
 
Your task:
 
1. Interpret the priorities and concerns in the business message.
2. Assign importance weights to each KPI (numeric values between 0 and 100) according to the business goals.
3. Ensure the final weights:
   - Sum exactly to 100
   - Include a weight for every KPI listed above
   - Are numeric and appropriate for the described goals
4. Return:
   - A JSON object called "weights" with weights for each KPI.
   - A JSON object called "explanations" with a 1-2 sentence justification for each weight.
5. Optionally include an ordered weight vector for downstream systems.
 
Always follow the user's business intent — even if they don’t use technical language.
"""

natural_language_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_nl),
    ("human", "Here is the current KPI list:\n\n{kpi_list}\n\n"
              "Now here's the business requirement, written by a stakeholder:\n\n\"\"\"\n{business_requirement}\n\"\"\"\n\n"
              "Please generate a set of KPI weights that reflect these goals, explain your reasoning, and return the result as a JSON.")
])


In [34]:
from langchain_openai import ChatOpenAI

# Define your JSON schema
kpi_schema = {
    "title": "KPIWeightAnalysis",
    "type": "object",
    "properties": {
        "weights": {
            "type": "object",
            "description": "A dictionary mapping each KPI name to a float weight out of 100",
            "additionalProperties": {
                "type": "number"
            }
        },
        "explanations": {
            "type": "object",
            "description": "A dictionary with KPI names and brief justifications for their weights",
            "additionalProperties": {
                "type": "string"
            }
        },
        "ordered_weights": {
            "type": "array",
            "description": "List of weights in the same order as given KPIs",
            "items": {
                "type": "number",
                "minimum": 0,
                "maximum": 100
            }
        }
    },
    "required": ["weights", "explanations", "ordered_weights"]
}


In [35]:
def natural_language_kpi_node(state: dict):
    kpi_list = state["kpi_list"]
    business_requirement = state["business_requirement"]

    messages = natural_language_prompt.format_messages(
        kpi_list=kpi_list,
        business_requirement=business_requirement
    )

    # Use the structured output with JSON schema
    llm = ChatOpenAI(model="gpt-4o-mini").with_structured_output(schema=kpi_schema)
    structured_result = llm.invoke(messages)

    return {"adjusted_weights_natural": structured_result}


In [36]:
from langgraph.graph import StateGraph

graph_builder = StateGraph(dict)
graph_builder.add_node("NaturalLangWeightAdjust", natural_language_kpi_node)
graph_builder.set_entry_point("NaturalLangWeightAdjust")
graph_builder.set_finish_point("NaturalLangWeightAdjust")

graph_nl = graph_builder.compile()


In [38]:
input_natural = {
    "kpi_list": '{ "Avg_Maintenance_Working_Time_Hours": 0.153279303, "Total_Cost": 0.158574872, "Unscheduled_Repairs": 0.143271754, "MTBF_Days": 0.159661443, "Total_PM": 0.139025173, "Avg_Distance_Post_Maint": 0.092867917, "Avg_Downtime_Hours": 0.153319536 }',
    "business_requirement": """
Downtime is killing productivity — we need to reduce it urgently.
Repairs that happen without warning are expensive and hurt trust with clients.
We also want to focus more on preventive maintenance to reduce those breakdowns.
Of course, total cost still matters.
We're less concerned about how long the work takes or how far vehicles go afterward.
"""
}

result = graph_nl.invoke(input_natural)
result
# print(json.dumps(result["adjusted_weights_natural"], indent=2))

{'adjusted_weights_natural': {'weights': {'Avg_Maintenance_Working_Time_Hours': 5,
   'Total_Cost': 25,
   'Unscheduled_Repairs': 20,
   'MTBF_Days': 20,
   'Total_PM': 20,
   'Avg_Distance_Post_Maint': 5,
   'Avg_Downtime_Hours': 5},
  'explanations': {'Avg_Maintenance_Working_Time_Hours': 'Given the emphasis on reducing downtime, the time taken for maintenance is less critical and receives the lowest weight.',
   'Total_Cost': 'Maintaining financial considerations is essential so this KPI gets a moderate weight to ensure cost management is considered.',
   'Unscheduled_Repairs': 'Given the urgent need to reduce unexpected repairs, this KPI is heavily weighted to reflect its importance in maintaining productivity and client trust.',
   'MTBF_Days': 'This KPI serves as a measure of vehicle reliability; increased MTBF is crucial for minimizing breakdowns, thus it holds high importance.',
   'Total_PM': 'Focusing on preventive maintenance is essential to tackle breakdowns effectively, ju